## Introduction

In this exercise you will work with binary data and files containing such data. In principle, any file which is not a text file may be called a binary file. The smallest entity in a text file is a character. Historically, a single character is an ASCII character and has the length of a byte (8 bits). Several characters are then structured into lines which are separated by line ends. A binary file lacks this structure. While in a binary file the bits are still grouped into bytes, each individual bit or arbitrarily long sequences of bits may represent a data entry. This means that you have to know the internal data layout of a binary file.


## Reading binary image files

Many different binary file layouts exist such as JPEG images or Microsoft Office data files. Often, as is the case for Microsoft Office data files, the internal structure is secret to a company. We will be dealing today with Netpbm image files which are in an open format and are portable. There are 6 types of Netpbm files identified by the “magic number” in the first line of the file:


| Magic number | Type | Usual extension | Encoding |
| --- | --- | --- | --- |
| P1 | Bitmap | .pbm | Ascii text |
| P2 | Graymap | .pgm | Ascii text |
| P3 | Pixmap | .ppm | Ascii text |
| P4 | Bitmap | .pbm | Binary |
| P5 | Graymap | .pgm | Binary |
| P6 | Pixmap | .ppm | Binary |

To get started, download the files 'binary_data.txt' and 'ppm_bin_example_p6.ppm' from Canvas, and upload them to Google Drive. Make sure to mount your Google Drive before proceeding!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

There are plenty of libraries that provide binary image parsers, such as [imageio](https://imageio.github.io/) and [pillow](https://pillow.readthedocs.io/en/3.0.x/index.html), but here we're going to do it the hard way to demonstrate how to handle binary data. The following cell contains a set of functions designed to load a binary .ppm file (magic number P6), but the same principles can be applied to any binary data as long as you know the format of the data.

The following cell contains a function that takes a file handle (i.e. the object created when we call open() on a file name) to parse the header of a binary .ppm file. Notice that, just like a raw text file, we can still read binary data line-by-line. In this case, python will simply read the binary data up to the first newline, '\n'. Spend some time reading through this code, and convince yourself that you understand what's going on. What happens if you use a different encoding, or don't decode the data at all?

In [ ]:
def parse_header(file_handle):
	# Read a line from the image file
    # Notice that we need to tell Python the encoding of the data before it can convert it to a text string
    # In this case, 'ascii' or 'utf-8' (unicode) both work
    line = file_handle.readline()
    line = line.decode('utf-8')

    # Now that we have text, we can check for the right magic number
    if (line == 'P6\n'):
        print('Reading a PPM file, format P6.')
    else:
        print('Exiting with error: not a P6 format PPM file.')
        exit(1)
    
    # Now we continue through the file until we've extracted the rest of the header
    # We know that line starting with '#' are comments
    # So we can just read line-by-line until we reach a line that doesn't start with '#'
    # Note that file_handle iterates over the opened file, so we can pick up where we left off
    line = '#'
    while line.startswith('#'):
        line = file_handle.readline()
        line = line.decode('utf-8')
    
    # The first line after the comment block is the (x, y) resolution of the image
    # Note that we already have the first non-comment line stored using the 'line' variable from our loop
    line = line.strip().split()
    x = int(line[0])
    y = int(line[1])
    # The second line after the comment block is the bit depth of the image
    line = file_handle.readline()
    line = line.decode('utf-8')
    depth = int(line)

    return(int(x), int(y), int(depth))

# passing 'rb' to open() tells python that we want to open a binary file ('b') in read mode ('r')
with open('/content/drive/My Drive/ppm_bin_example_p6.ppm','rb') as f:
    parse_header(f)

In the .ppm file, each pixel is represented by three integers in the range [0, 255\]. Since this range of integers can be represented by 8 bits, or one byte, each RGB pixel is stored in the file as three bytes. To decode the image, we load the binary data into memory, then iterate through the data one byte at a time, decoding that byte to obtain an 8-bit integer. Every three decoded integers corresponds to one pixel.

Here is a function that iterates over a 'chunk' of bytes, and prints each pixel as it is decoded. We then use a loop to iterate over the binary data one chunk at a time, decoding each chunk and printing the pixel values.

In [ ]:
def parse_binary_data(bytes):

	# A complete pixel is three bytes. We'll use a list to keep track of the current pixel
	rgb_pixel = ['-','-','-']

	# We'll use a counter to keep track of which pixel we're currently decoding
    # current_colour=0 corresponds to red, 1 corresponds to green, 2 corresponds to blue
	current_colour = 0
	
	# Go through the bytes in the chunk. Three bytes make a pixel, so refresh rgb_pixel every 3 iterations
	for byte in bytes:

		#at the start of a new pixel, reset all RGB values to 0
		if current_colour == 0 :
			rgb_pixel = ['-', '-', '-']

		# Convert the byte char we read to an int between 0 and 255
		rgb_pixel[current_colour] = byte

		#keep track of whether we're on R, G, or B by incrementing current_colour mod 3
		current_colour = current_colour + 1
		if current_colour == 3:
			current_colour = 0 

		#print some progress to make output more readable
		print("updating rgb_pixel: ", rgb_pixel)
		print('------------------------------')
		if current_colour == 0:
			print('=============== COMPLETE PIXEL ===============')

Here we use our two functions to parse the complete image file. First, we extract the header to tell us the format of the file, then, continuing from the point in the file where we stopped parsing the header, we read and decode the image in chunks, printing our progress as we go. Note that open() accepts a 'size' argument in both text and binary mode. In text mode, this causes open() to return up to the specified number of characters, while in binary mode it instead returns up to the specified number of bytes.

In [ ]:
with open('/content/drive/My Drive/ppm_bin_example_p6.ppm','rb') as f:

    # Parse theimage header to get the number of columns, number of rows, and bit depth of the image
    (x_rez, y_rez, bit_depth) = parse_header(f)
    print(f'x resolution {x_rez}; y resolution {y_rez}; bit depth {bit_depth}')

    # Set the size of our memory buffer - this is the maximum number of bytes we want to read in at a time
    buffer_length = 3

    while True:

        # Read the next chunk - up to buffer_length bytes
        print(f'Loading {buffer_length} bytes into memory buffer.')
        bytes = f.read(buffer_length) 

        # Terminate when there are no more bytes in the file
        # Note that an empty bytestring evaluates as False
        # We can use this to check whether we're done
        if not bytes:
            break
        
        parse_binary_data(bytes)



Before going any further, take a moment to play with this code. What happens if you set the buffer_length to a number greater the number of bytes in the image (3*3*2 = 18)? What happens if you set the buffer_length to a number less than the total number of bytes, and not a multiple of 3?

## Exercises

### Exercise 1

Starting from the code above, write a function to store each of the three colour channels (R, G, B) in the image as a two-dimensional array, and return these three arrays.

### Exercise 2
Print only the green colour channel of the image.

### Exercise 3

Write the green colour channel of the image to a new .ppm or .pgm file. Download your new file and open it in an image editor to verify that your file contains the correct pixel values.

## Converting binary data to text

We’ll now parse some 'fake' binary data into characters to understand how to convert from binary, to ASCII, and finally to human-readable characters. The file 'binary_data.txt' contains some 'binary' data represented as a string of 0s and 1s. Take a look in the file to get an idea for what the stream of binary data looks like to the computer.

For 8-bit binary data, every eight bits in the file corresponds to a single value to be decoded. This is why it's necessary to know the encoding of a binary data file - if somebody gives you some 32-bit data, but your system assumes everything is 64-bit, you're going to get some very peculiar results!

For example, the first eight bits of the file are 01010100.  We can convert that to ASCII by taking,

$0\times2^7+1\times2^6+0\times2^5+1\times2^4+0\times2^3+1\times2^2+0\times2^1+0\times2^0 =64+16+4=84$,

which corresponds to the character 'T'.



### Exercise 4

Starting from the skeleton code below, write a function to convert a binary byte to an ASCII integer value, and a function to convert an ASCII integer value to a human-readable character. Remember, you'll need to covert the 'byte' string into eight ints representing the individual bits, then convert the bits into the corresponding 8-bit integer.

In [ ]:
def byteToAscii(b):
    # TODO implement me!
    pass

def asciiToChar(a):
    # TODO implement me!
    pass

test_byte = '01010100'
test_ascii = 84

# Uncomment when you're ready to test a function
#byte_to_ascii(test_byte)
#byte_to_ascii(test_ascii)

Once you're confident your functions are behaving as expected, try running the cell below to convert the 'binary' data into human-readable text.

In [ ]:
# The file contains a single line
# We read the entire line into a single string (pretend it's binary!)
with open('/content/drive/My Drive/binary_data.txt') as f:
    binary_string = f.readline().strip()

# Loop over the 'binary' string, one 'byte' (i.e. 8 'bits') at a time
for i in range(0, len(binary_string), 8):

	# Get the nect byte
	byte = binary[i:i+8]

	#convert the byte to ascii
	ascii_byte = byte_to_ascii(byte)

	#convert the ascii to a character, which is human readable (so we can print it)
	character = ascii_to_char(ascii_byte)
	print(character, end="")